<a href="https://colab.research.google.com/github/ameyanator/data-science/blob/main/template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')
data = pd.read_csv('/content/drive/MyDrive/Datasets/Housing Dataset Kaggle/train.csv')
data.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 69)
print(X_train.head())

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
869    870          60       RL         80.0     9938   Pave   NaN      Reg   
1324  1325          20       RL         75.0     9986   Pave   NaN      Reg   
384    385          60       RL          NaN    53107   Pave   NaN      IR2   
769    770          60       RL         47.0    53504   Pave   NaN      IR2   
341    342          20       RH         60.0     8400   Pave   NaN      Reg   

     LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence  \
869          Lvl    AllPub  ...           0        0    NaN  GdPrv   
1324         Lvl    AllPub  ...           0        0    NaN    NaN   
384          Low    AllPub  ...           0        0    NaN    NaN   
769          HLS    AllPub  ...         210        0    NaN    NaN   
341          Lvl    AllPub  ...           0        0    NaN    NaN   

     MiscFeature MiscVal MoSold  YrSold  SaleType  SaleCondition  
869          NaN       0      6    20

In [4]:
X_test=pd.read_csv('/content/drive/MyDrive/Datasets/Housing Dataset Kaggle/test.csv')
print(X_test.head())
print("Initial X_train columns:", X_train.columns)
print("Initial X_valid columns:", X_valid.columns)
print("Initial X_test columns:", X_test.columns)

     Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub  ...         120        0    NaN  MnPrv         NaN   
1         Lvl    AllPub  ...           0        0    NaN    NaN        Gar2   
2         Lvl    AllPub  ...           0        0    NaN  MnPrv         NaN   
3         Lvl    AllPub  ...           0        0    NaN    NaN         NaN   
4         HLS    AllPub  ...         144        0    NaN    NaN         NaN   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  
0       0      6

In [5]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and
                    X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if
                X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
print("Selected columns:", my_cols)
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test = X_test[my_cols].copy()
print("X_train selected columns:", X_train.columns)
print("X_valid selected columns:", X_valid.columns)
print("X_test selected columns:", X_test.columns)

Selected columns: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 

In [6]:
# find columns which have data in train set but not validation set
def seperate_good_bad_columns_categorical(X_train, X_valid):
  # Categorical columns in the training data
  object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

  # Columns that can be safely ordinal encoded
  good_label_cols = [col for col in object_cols if
                    set(X_valid[col]).issubset(set(X_train[col]))]

  # Problematic columns that will be dropped from the dataset
  bad_label_cols = list(set(object_cols)-set(good_label_cols))

  print('Categorical columns that will be ordinal encoded:', good_label_cols)
  print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)
  return good_label_cols, bad_label_cols

In [7]:
def prepare_data_for_ordinal_encoding(X_train, X_valid, X_test):
  # Select numerical columns
  numerical_cols = [cname for cname in X_train.columns if
                  X_train[cname].dtype in ['int64', 'float64']]
  # this step is required for ordinal encoding
  cols = seperate_good_bad_columns_categorical(X_train, X_valid)
  good_cols, bad_cols = cols[0], cols[1]
  my_cols = numerical_cols + good_cols
  X_train = X_train[my_cols].copy()
  X_valid = X_valid[my_cols].copy()
  X_test = X_test[my_cols].copy()
  print("Columns to be used:", my_cols)
  return X_train, X_valid, X_test, numerical_cols, good_cols

In [ ]:
new_data = prepare_data_for_ordinal_encoding(X_train, X_valid, X_test)
X_train, X_valid, X_test, numerical_cols, categorical_cols = new_data[0], new_data[1], new_data[2], new_data[3], new_data[4]

print("X_train cols after preparation:", X_train.columns)
print("X_valid cols after preparation:", X_valid.columns)
print("X_test cols after preparation:", X_test.columns)
print("Numerical columns:", numerical_cols)

def build_and_execute_pipeline(numerical_imputer_strategy, categorical_imputer_strategy, n_estimators):
  print(numerical_imputer_strategy, categorical_imputer_strategy, n_estimators)
  # Preprocessing for numerical data
  numerical_transformer = SimpleImputer(strategy=numerical_imputer_strategy)

  # Preprocessing for categorical data
  categorical_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy=categorical_imputer_strategy)),
      ('ordinal', OrdinalEncoder())
  ])

  # Bundle preprocessing for numerical and categorical data
  preprocessor = ColumnTransformer(
      transformers=[
          ('num', numerical_transformer, numerical_cols),
          ('cat', categorical_transformer, categorical_cols)
      ])

  # Define model
  model = RandomForestRegressor(n_estimators=n_estimators, random_state=69)

  # Bundle preprocessing and modeling code in a pipeline
  clf = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', model)
                      ])
  clf.fit(X_train, y_train)
  preds = clf.predict(X_valid)

  print('MAE:', mean_absolute_error(y_valid, preds))

numerical_imputer_strategies = ['mean', 'median', 'most_frequent', 'constant']
categorical_imputer_strategies = ['most_frequent', 'constant']
n_estimators = [10, 100, 200, 300, 500]

for numerical_imputer_strategy in numerical_imputer_strategies:
  for categorical_imputer_strategy in categorical_imputer_strategies:
    for n_estimator in n_estimators:
      print('Calling pipeline with numerical_imputer_strategy: {} and categorical_imputer_strategy:{}'.format(numerical_imputer_strategy, categorical_imputer_strategy))
      print('n_estimator: {}'.format(n_estimator))
      build_and_execute_pipeline(numerical_imputer_strategy, categorical_imputer_strategy, n_estimator)
    print("-------------------")
  print("****************")

Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'BldgType', 'HouseStyle', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['RoofMatl', 'Electrical', 'Condition2', 'RoofStyle', 'Heating']
Columns to be used: ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'F